In [1]:
import numpy as np
import pandas as pd
import time
from datetime import datetime
from glob import glob
import os
from math import ceil

import ccxt

from tp_utils.data_provider import get_symbols, market_to_symbol, load_data_from_exchange
from tp_config import *

pd.set_option('display.precision', 9)

In [2]:
pair = 'USDT'
timeframes = {'1m':1, '3m':3, '5m':5, '15m':15}
#timeframes = {'30m':30, '1h':60} #'2h':120, '4h':240, '6h':480, '12h':960, '1d':1920}
#timeframes = {'1h':60}# '2h':120, '4h':240, '6h':480, '12h':960, '1d':1920}
timeframes = {'4h':240, '6h':480, '12h':960, '1d':1920}
timeframes = {'4h':240}
exchange = ccxt.bybit()
symbol_list = get_symbols(exchange, pair)

symbols = [x for x in symbol_list if ":" not in x]

In [3]:
index_assets = pd.read_csv('Portfolio/index_assets.csv')['asset'].tolist()
index_low_assets = pd.read_csv('Portfolio/index_low_assets.csv')['asset'].tolist()
full_assets = index_assets + index_low_assets

markets = [x.replace('/','-') for x in symbol_list if x.split('/')[0] != 'BTC']

main = ['USDT']

assets = full_assets
#markets = main
markets = [s + '-' + pair for s in assets] 

In [56]:
markets = [x.replace('/','-') for x in symbols]
data_path = DATA_PATH_CRYPTO + '/' + pair

now = exchange.milliseconds()
to_timestamp = now
start_timestamp = exchange.parse8601('2023-10-01 00:00:00')

### 0. Data from exchange

In [59]:
pair = "USDC"
data_path = DATA_PATH_CRYPTO + '/' + pair
markets = ["BTC-USDC", "SOL-USDC"]
timeframes = {'5m':5, '15m':15}

In [60]:
load_data_from_exchange(exchange, markets, timeframes, start_timestamp,  data_path, 10)

1696118400000 2023-10-01 04:00:00
BTC-USDC
5m


D:\Code\Projects\QuantMod\tp\tp_utils\data_provider.py:222: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df_old, df_new])


2024-03-18 21:00:00
15m


D:\Code\Projects\QuantMod\tp\tp_utils\data_provider.py:222: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df_old, df_new])


2024-03-18 21:00:00
SOL-USDC
5m


D:\Code\Projects\QuantMod\tp\tp_utils\data_provider.py:222: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df_old, df_new])


2024-03-18 21:00:00
15m
2024-03-18 21:00:00
Done
Time taken = 0.073  hours


D:\Code\Projects\QuantMod\tp\tp_utils\data_provider.py:222: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df_old, df_new])


In [32]:
ohlcvs =  exchange.fetch_ohlcv ('JUP/USDT', 240, start_timestamp)
ohlcvs

[]

### PyBit

In [47]:
import pybit

In [54]:
from pybit.unified_trading import HTTP
session = HTTP()
print(session.get_kline(
    category="spot",
    symbol="SOLUSDC",
    interval=5,
    start=1706716267000,
))

{'retCode': 0, 'retMsg': 'OK', 'result': {'category': 'spot', 'symbol': 'SOLUSDC', 'list': [['1706775900000', '94.61', '94.86', '94.55', '94.68', '565.023', '53528.91517'], ['1706775600000', '94.61', '94.81', '94.57', '94.61', '469.24', '44433.79064'], ['1706775300000', '94.64', '94.78', '94.52', '94.61', '622.504', '58896.23589'], ['1706775000000', '94.65', '94.83', '94.56', '94.64', '157.219', '14887.89186'], ['1706774700000', '95.14', '95.18', '94.65', '94.65', '135.56', '12871.62796'], ['1706774400000', '94.86', '95.15', '94.73', '95.14', '219.929', '20876.47782'], ['1706774100000', '94.97', '95.07', '94.75', '94.86', '174.637', '16578.03578'], ['1706773800000', '95.59', '95.59', '94.96', '94.97', '698.033', '66487.94296'], ['1706773500000', '95.37', '95.7', '95.35', '95.59', '619.808', '59198.6434'], ['1706773200000', '95.5', '95.51', '95.3', '95.37', '225.353', '21501.49672'], ['1706772900000', '95.4', '95.56', '95.29', '95.5', '563.572', '53762.43589'], ['1706772600000', '95.26'

In [48]:
session = HTTP(testnet=True)
print(session.get_instruments_info(
    category="linear",
    symbol="JUPUSDT",
))

{'retCode': 0, 'retMsg': 'OK', 'result': {'category': 'linear', 'list': [{'symbol': 'JUPUSDT', 'contractType': 'LinearPerpetual', 'status': 'Trading', 'baseCoin': 'JUP', 'quoteCoin': 'USDT', 'launchTime': '1706716267000', 'deliveryTime': '0', 'deliveryFeeRate': '', 'priceScale': '4', 'leverageFilter': {'minLeverage': '1', 'maxLeverage': '50.00', 'leverageStep': '0.01'}, 'priceFilter': {'minPrice': '0.0001', 'maxPrice': '199.9998', 'tickSize': '0.0001'}, 'lotSizeFilter': {'maxOrderQty': '30000', 'minOrderQty': '1', 'qtyStep': '1', 'postOnlyMaxOrderQty': '30000', 'maxMktOrderQty': '15000'}, 'unifiedMarginTrade': True, 'fundingInterval': 240, 'settleCoin': 'USDT', 'copyTrading': 'none', 'upperFundingRate': '0.006', 'lowerFundingRate': '-0.006'}], 'nextPageCursor': ''}, 'retExtInfo': {}, 'time': 1710779933743}
